<a href="https://colab.research.google.com/github/bhaskarachalla/Developing_models_on_own_dataset/blob/master/LoRA(DistilGPT)_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q datasets
! pip install -q bitsandbytes
! pip install -q peft #parameter efficient fine tuning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 7.7 MB/s eta 0:00:00


In [15]:
from typing import Dict, List
from datasets import Dataset, load_dataset, disable_caching
disable_caching()
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from torch.utils.data import Dataset
from IPython.display import Markdown
import pandas as pd

In [16]:
dataset = load_dataset("fka/awesome-chatgpt-prompts", split = 'train')
print(dataset)

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

prompts.csv:   0%|          | 0.00/84.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/170 [00:00<?, ? examples/s]

Dataset({
    features: ['act', 'prompt'],
    num_rows: 170
})


In [17]:
prompt_template = """Below is an instruction that describes a task, Write a response that approporiately complete the request. Instruction: {instruction} \n Response: """
answer_template = """{response}"""

# Creating a function to add keys in the dictionary for prompt, answer and whole text

def _add_text(rec):
  instruction = rec["prompt"]
  response = rec["act"]
  if not instruction:
    raise ValueError(f"Expected an instructions in: {rec}")
  if not response:
    raise ValueError(f"Expected a response in: {rec}")

  rec["prompt"] = prompt_template.format(instruction = instruction)
  rec["answer"] = answer_template.format(response = response)
  rec["text"] = rec["prompt"] + rec["answer"]

  return rec

dataset = dataset.map(_add_text)
print(dataset[0])

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

{'act': 'An Ethereum Developer', 'prompt': 'Below is an instruction that describes a task, Write a response that approporiately complete the request. Instruction: Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation. \n Response: ', 'answer': 'An Ethereum Developer', 'text': 'Below is an instruction that describes a task, Write a response that approporiately complete the request. Instruction: Imagine you are an experienced Ethereum developer tasked with creating a smart 

In [18]:
model_id = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map = "auto",
    load_in_8bit = True,
    torch_dtype = torch.float16
)

model.resize_token_embeddings(len(tokenizer))
print(model)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Linear8bitLt(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear8bitLt(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear8bitLt(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear8bitLt(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, element

In [19]:
torch.cuda.empty_cache()

A data collator is a utility function used in sequence-to-sequence (seq2seq) algorithms to prepare and format the data for training. Its primary purpose is to take a batch of input data and corresponding target data, and return a formatted batch that can be fed into the seq2seq model.

The data collator typically performs the following tasks:

1. Padding: Pads the input sequences to have the same length, usually the maximum length in the batch.
2. Masking: Creates masks to indicate which parts of the input sequences are padding tokens.
3. Tokenization: Converts the input sequences into token IDs.
4. Batching: Combines the input sequences, masks, and token IDs into a single batch.

The formatted batch is then passed to the seq2seq model for training. The data collator is usually implemented as a Python function or a PyTorch Dataset class.


In [20]:
from functools import partial
import copy
from transformers import DataCollatorForSeq2Seq

MAX_LENGTH = 256

def _preprocess_batch(batch: Dict[str, List]) :
  model_inputs = tokenizer(batch["text"], max_length=MAX_LENGTH, truncation = True, padding = 'max_length')
  model_inputs["labels"] = copy.deepcopy(model_inputs['input_ids'])

  return model_inputs

_preprocessing_function = partial(_preprocess_batch)
encode_small_dataset = dataset.map(
    _preprocessing_function,
    batched = True,
    remove_columns = ["prompt", "act", "prompt", "answer"]
)


processed_dataset = encode_small_dataset.filter(lambda rec: len(rec["input_ids"]) <= MAX_LENGTH)

# SPillting dataset
split_dataset = processed_dataset.train_test_split(test_size = 14, seed = 42)
print(split_dataset)

data_collator = DataCollatorForSeq2Seq(model = model, tokenizer=tokenizer, max_length = MAX_LENGTH, pad_to_multiple_of = 8, padding = "max_length")

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

Filter:   0%|          | 0/170 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 156
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14
    })
})


In [21]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model # Importing LoraConfig from peft instead of transformers
#from transformers import BartForCausalLM

In [22]:
LORA_R = 256
LORA_ALPHA = 512
LORA_DROPOUT = 0.05

# DEFINE LoRA Config

lora_config = LoraConfig(
    r = LORA_R, # the dimnesions low rank matrices
    lora_alpha = LORA_ALPHA, # Scaling factor for the weight matrices
    lora_dropout = LORA_DROPOUT, # dropout probability of the LoRA layer
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules=['transformer.h.0.attn.c_attn', 'transformer.h.0.attn.c_proj',
                     'transformer.h.0.mlp.c_fc', 'transformer.h.0.mlp.c_proj',
                     'transformer.h.1.attn.c_attn', 'transformer.h.1.attn.c_proj',
                     'transformer.h.1.mlp.c_fc', 'transformer.h.1.mlp.c_proj',
                     'transformer.h.2.attn.c_attn', 'transformer.h.2.attn.c_proj',
                     'transformer.h.2.mlp.c_fc', 'transformer.h.2.mlp.c_proj',
                     'transformer.h.3.attn.c_attn', 'transformer.h.3.attn.c_proj',
                     'transformer.h.3.mlp.c_fc', 'transformer.h.3.mlp.c_proj',
                     'transformer.h.4.attn.c_attn', 'transformer.h.4.attn.c_proj',
                     'transformer.h.4.mlp.c_fc', 'transformer.h.4.mlp.c_proj',
                     'transformer.h.5.attn.c_attn', 'transformer.h.5.attn.c_proj',
                     'transformer.h.5.mlp.c_fc', 'transformer.h.5.mlp.c_proj',]
)

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 18,874,368 || all params: 100,786,944 || trainable%: 18.7270


In [23]:
from transformers import TrainingArguments, Trainer
import bitsandbytes
# define the training arguments first.
EPOCHS = 3
LEARNING_RATE = 2e-5
MODEL_SAVE_FOLDER_NAME = "DistilGPT2_LORA"
training_args = TrainingArguments(
                    output_dir=MODEL_SAVE_FOLDER_NAME,
                    overwrite_output_dir=True,
                    fp16=True, #converts to float precision 16 using bitsandbytes
                    per_device_train_batch_size=1,
                    per_device_eval_batch_size=1,
                    learning_rate=LEARNING_RATE,
                    num_train_epochs=EPOCHS,
                    logging_strategy="epoch",
                    evaluation_strategy="epoch",
                    save_strategy="epoch",
)
# training the model
trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=split_dataset['train'],
        eval_dataset=split_dataset["test"],
        data_collator=data_collator,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
# only saves the incremental 🤗 PEFT weights (adapter_model.bin) that were trained, meaning it is super efficient to store, transfer, and load.
trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)
# save the full model and the training arguments
trainer.save_model(MODEL_SAVE_FOLDER_NAME)
trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbyt

Epoch,Training Loss,Validation Loss
1,2.259300,1.192876
2,1.325000,1.100989
3,1.258000,1.085249


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cp

In [24]:
# Function to format the response and filter out the instruction from the response.
def postprocess(response):
    messages = response.split("Response:")
    if not messages:
        raise ValueError("Invalid template for prompt. The template should include the term 'Response:'")
    return "".join(messages[1:])
# Prompt for prediction
inference_prompt = "Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation."
# Inference pipeline with the fine-tuned model
inf_pipeline =  pipeline('text-generation', model=trainer.model, tokenizer=tokenizer, max_length=256, trust_remote_code=True)
# Format the prompt using the `prompt_template` and generate response
response = inf_pipeline(prompt_template.format(instruction=inference_prompt))[0]['generated_text']
# postprocess the response
formatted_response = postprocess(response)
formatted_response

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

' '

In [25]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['the', 'cat',"is","sitting","on","the","mat"]]
test = ["on",'the',"mat","is","a","cat"]
score = sentence_bleu(  reference, test)
print(score)

5.5546715329196825e-78


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
